In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [7]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [0]:
my_log_reg_model = LogisticRegression()

In [0]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [0]:
log_summary = fitted_logreg.summary

In [12]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [0]:
lr_train,lr_test = my_data.randomSplit([0.7,0.3])

In [0]:
final_model = LogisticRegression()

In [0]:
fit_final = final_model.fit(lr_train)

In [0]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [18]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[23.4706906845332...|[0.99999999993590...|       0.0|
|  0.0|(692,[100,101,102...|[10.7816138335672...|[0.99997922239037...|       0.0|
|  0.0|(692,[122,123,124...|[20.3754981196014...|[0.99999999858409...|       0.0|
|  0.0|(692,[124,125,126...|[38.1505699977623...|[1.0,2.7003370083...|       0.0|
|  0.0|(692,[124,125,126...|[32.7128113696880...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[22.4460510537293...|[0.99999999982143...|       0.0|
|  0.0|(692,[124,125,126...|[24.1401724547417...|[0.99999999996718...|       0.0|
|  0.0|(692,[126,127,128...|[19.0325586282103...|[0.99999999457668...|       0.0|
|  0.0|(692,[126,127,128...|[34.2432786950352...|[0.99999999999999...|       0.0|
|  0.0|(692,[126

### Using Evaluator object

In [0]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                  MulticlassClassificationEvaluator)

In [0]:
my_eval = BinaryClassificationEvaluator()

In [0]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [22]:
my_final_roc

1.0